# Programando con Python Spark (PySpark)

Recordando:
- El driver program accesa al ambiente de Spark mediante un objeto SparkContext
- El tipo de dato y concepto clave en Spark es el dataset llamados RDD
- Cargamos datos en un RDD y hacemos operaciones

## ¡Nuestro primer programa!

In [ ]:
!wget https://raw.githubusercontent.com/apache-spark/spark/master/README.md

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext(master = "local[*]")

In [ ]:
lineas = sc.textFile("README.md", 4)

In [ ]:
lineas

In [ ]:
lineas.count()

In [ ]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [ ]:
lineasPython

In [ ]:
lineasPython.take(4)

----

## RDD

**R**esilient **D**istributed **D**ateset (**RDD**)

Recordando y agregando:
- Contiene **Datos distribuidos** mediante **particiones** (de Workers)
- Habilita **operaciones** para su **ejecución en paralelo**
- Son **inmutables**
- **En caso de pérdida, la computación ejecutada se re-ejecuta**


Tres maneras de crear RDDs:

- Mediante un dataset externo:

In [ ]:
lineas = sc.textFile("README.md", 4)

In [ ]:
lineas

- Distribuyendo una colección:

In [ ]:
lineas = sc.parallelize([1, 2, 3])

In [ ]:
lineas

- Transformando un RDD existente:

In [ ]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [ ]:
lineasPython

### Operaciones sobre RDDs

#### Transformaciones

Crean un nuevo RDD a partir de otro previo.

P. ej.:
*map()*


#### Acciones

Corre/ejecuta/computa un resultado basado en un RDD existente.

P. ej.:
*count()*

## Programación Funcional con Python

- Muchas transformaciones y algunas acciones esperan una función
- En algunos casos, pueden ser funciones para operaciones más complejas
- Para funciones simples, una expresión lambda es conveniente:
```python
>>> lambda line: “Python” in line
```


### map()

- Lee un elemento a la vez
- Toma un valor, crea un nuevo valor


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4])

In [ ]:
rdd.map(lambda x: x * 2)

In [ ]:
rdd.map(lambda x: x * 2).collect()

### filter()

- Lee un elemento a la vez
- Evalua cada elemento
- Regresa los elementos que pasan el filtro  (filtro)

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4])

In [ ]:
BBB = rdd.filter(lambda x: x % 2 == 0)

In [ ]:
BBB.collect()

In [ ]:
rdd.filter(lambda x: x % 2 == 0).collect()

### flatMap()

Produce multiples elementos por cada elemento de entrada

In [ ]:
rdd = sc.parallelize([1,2,3])

In [ ]:
rdd.map(lambda x: [x, x * 2]).take(2)

In [ ]:
rdd.flatMap(lambda x: [x, x * 2])

# Transformations are lazy!
## Featuring: Lazy evaluation!! 🔥🙈

[Haskell Lazy Evaluation](https://wiki.haskell.org/Lazy_evaluation):

>Lazy evaluation is a method to evaluate a Haskell program. It means that expressions are not evaluated when they are bound to variables, but their evaluation is deferred until their results are needed by other computations. In consequence, arguments are not evaluated before they are passed to a function, but only when their values are actually used. 

[The Incomplete Guide to Lazy Evaluation (in Haskell)](https://hackhands.com/guide-lazy-evaluation-haskell/):

> Originally, I wanted to write a complete guide to lazy evaluation, but then.

>Lazy evaluation is the most widely used method for executing Haskell program code on a computer. It determines the time and memory usage of Haskell programs, and it allows new and powerful ways to write modular code. To make full use of purely functional programming, a good understanding of lazy evaluation is very helpful.


Un RDD solo es ejecutado cuando las acciones corren sobre el mismo:

In [ ]:
lineas = sc.textFile("README.md", 4)
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [ ]:
lineasPython

In [ ]:
lineasPython.first()

Al usar la evaluación floja, Spark puede contener en memoria RDD que se procesa unicamente cuando se le requiere. Sin la necesidad de cargar a memoria todas las lineas conteniendo "Python".


💻🐍

## Acciones

- Las acciones causan transformaciones para ser almacenadas en RDDs nuevos
- También regresan resultados a ambas partes: el *driver* o un almacenamiento externo
- Los RDDs son re-calculados por cada acción que se les ejecuta
- Pueden ser almacenados para un uso posterior: `rdd.persist()`

### count()

Obtiene las instancias en el RDD:

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4])

In [ ]:
rdd

In [ ]:
rdd.count()

### collect()
- `collect()` recupera el RDD completo 🚒
- Útil para inspeccionar datasets pequeños de manera local y para unit-testing
- **LOS RESULTADOS DEBEN CABER EN LA MEMORIA DEL EQUIPO LOCAL**

In [ ]:
rdd = sc.parallelize([1, 2, 3])

In [ ]:
rdd

In [ ]:
rdd.takeOrdered(3)

### take(), takeSample(), first(), top(), takeOrdered() 

- ```take(n)``` regresa los primeros *n* elementos de un RDD
- ```take(n)``` puede obtener resultados sezgados. Su uso es adecuado solo para pruebas o debugging
- ```takeSample()``` como el nombre lo indica, es el más adecuado para tomar una muesta del dataset
- ```first(n)``` al igual que ```take(n)```, obtiene los primeros *n* elementos de un RDD
- ```top(), takeOrdered()``` como métodos más formales para obtener elementos ordenados de un RDD

### takeOrdered()

In [ ]:
rdd = sc.parallelize([5, 1, 3, 2])

In [ ]:
rdd.takeOrdered(4)

In [ ]:
rdd.takeOrdered(4, lambda n: -n)

### reduce()
Toma dos elementos del mismo tipo y regresa un nuevo elemento:

In [ ]:
rdd = sc.parallelize([1,2,3])

In [ ]:
rdd.reduce(lambda x, y:  x*y)

## Persistencia
- Spark re-calcula los RDDs cada vez que se llama a una acción:
    - Esto puede ser caro y también causar un tráfico innecesario desde el disco (lectura)
- Podemos evitar esto almacenando datos en caché con ```persist()```.

In [ ]:
lineas = sc.textFile("README.md", 4)

In [ ]:
lineas.count()

In [ ]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [ ]:
# Causa a Spark el recargar la variable "lineas" desde el disco 🙊
%time lineasPython.count()

### Aplicando persistencia

In [ ]:
lineas = sc.textFile("README.md", 4)

In [ ]:
lineas.persist() # Ahora, este RDD se mantiene en RAM

In [ ]:
lineas.count()

In [ ]:
lineasPython =  lineas.filter(lambda line :  "Python" in line)

In [ ]:
# Spark no volverá a hacer el cómputo para "lineas" cada vez que es usado
%time lineasPython.count()

## Construyendo un Pipeline de operaciones para RDDs



```python
>>> lineas = sc.textFile("README.md")
>>> lineas.map(...).filter(...).count(...)



>>> lineas = sc.textFile("README.md")
>>> (lineas
     .map(...)
     .filter(...)
     .count(...))
```

In [ ]:
lineas = sc.textFile("README.md")

In [ ]:
lineas.filter(lambda line :  "Python" in line).count()

---

### Ejercicio 01: Crear un nuevo RDD con la cadena "Hola Spark" e imprimirla en pantalla al obtener el primer elemento

### Ejercicio 02: Completar el siguiente bloque de código, para usar el archivo README.md e imprimir el numero de lineas y el conteo de palabras en el archivo

```python

# Crear un RDD a partir de un dataset
readme_rdd =  
# Imprimir en pantalla el num. de lineas del RDD
print('Conteo de lineas: ')
print()
print('Conteo de palabras: ')
palabras_lista = readme_rdd.flatMap(lambda linea: linea.split(" ")) \
                            . # map
                            .reduceByKey(lambda a, b: a + b) \
                            . # collect
  print(palabras_lista)

```

---
### Soluciones:

#### Ejercicio 01
```python
hola_spark_rdd = sc.parallelize('Hola Spark')
print(hola_spark_rdd.take(1))
print(hola_spark_rdd.collect())

>>> ['H']
>>> ['H', 'o', 'l', 'a', ' ', 'S', 'p', 'a', 'r', 'k']
```


#### Ejercicio 02: reduceByKey() y collect()
```python
readme_rdd = sc.textFile('README.md')
print('Conteo de lineas: ')
print(readme_rdd.count())
print('Conteo de palabras: ')
palabras_lista = readme_rdd.flatMap(lambda linea: linea.split(" ")) \
                            .map(lambda palabra: (palabra, 1)) \
                            .reduceByKey(lambda a, b: a + b) \
                            .collect()
  print(palabras_lista)
```

#### Word count con otro dataset y takeOrdered

Referencia:
```python
Sort by keys (ascending):

>>> RDD.takeOrdered(5, key = lambda x: x[0])

Sort by keys (descending):

>>> RDD.takeOrdered(5, key = lambda x: -x[0])

Sort by values (ascending):

>>> RDD.takeOrdered(5, key = lambda x: x[1])

Sort by values (descending):

>>> RDD.takeOrdered(5, key = lambda x: -x[1])
```



In [ ]:
!wget http://www.gutenberg.org/files/74/74-0.txt